# Train a v1 numerai model

## Goal
Train a simple numerai model on subsampled data to dip our toes in the modelling. As
an ancillary goal we will see how much does adding more data improves model performance.

## Steps
1. **[Subsample]** We will subsample the training and the validation data by sampling later eras more.
   We ignore old training eras.
2. **[Learning curve - num instances]** We will build a learning curve to see how much
   increasing the amount of training eras and by extension number of training rows matters.
3. **[Learning curve - num features]** We will do a similar analysis for different number of
   features.
4. **[XVal - Time split]** Finally we will test out the cross validation code shared by mdo.
5. **[4th era sampling]** We will run the code once for a dataset where only every 4th era
   is included. We will how much performance takes a hit this change accrues.

For scoring we will use the simple pearson correlation and not the era based
spearman score ie `numerai_score`.

## Observations
**[Learning curve - num instances]** Increasing the number of training samples does
 improve performance and we are very far from saturation.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import sklearn
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
import itertools as it

import pickle
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn import metrics
import metrics as nmr_metrics
from tqdm.notebook import tqdm
import shap
import cufflinks as cf

cf.set_config_file(theme='pearl', sharing='public', offline=True)

# Increase font size of matplotlib plots
plt.rcParams['figure.figsize'] = 16, 8
plt.rcParams['font.size'] = 16

In [ ]:
import modelling_utils as mu

## Load training data

Let's load the data and save a smaller version for quicker startup.

In [ ]:
TO_LOAD_SML_DATA = False
ONLY_4TH_ERA = False
MIN_TRAIN_ERA = 400

TRAIN_DATA = "../data/v4.1/train_int8.parquet"
VAL_DATA = "../data/v4.1/validation_int8.parquet"
# Subsampled training and validation data for faster iteration
SPLIT_DATA_4TH_ERA_FILE = "../data/v4.1/split_dfs_sml.pkl"
SPLIT_DATA_ALL_RECENT_ERA_FILE = "../data/v4.1/split_dfs_sml_4th_era.pkl"
SPLIT_DATA_FILE  = SPLIT_DATA_4TH_ERA_FILE if ONLY_4TH_ERA else SPLIT_DATA_ALL_RECENT_ERA_FILE

In [ ]:
if TO_LOAD_SML_DATA:
    split_dfs = mu.load_sampled_data(sampled_save_fl=SPLIT_DATA_FILE)
else:
    split_dfs = mu.sample_and_save_data(
        train_path=TRAIN_DATA,
        val_path=VAL_DATA,
        train_min_erano=300,
        sampled_save_fl=SPLIT_DATA_FILE,
        only_4th_erano=ONLY_4TH_ERA,
    )

In [ ]:
train_df, val_df = split_dfs["train"], split_dfs["val"]
val_df = val_df[~val_df["target"].isnull()]
print(f"{train_df.shape=}")
display(train_df.head(2))
display(val_df.head(2))

#### Distribution of eras and target

In [ ]:
features = mu.get_features(train_df)
target = "target"
eras = train_df.erano
print(f"{len(features)=}\n")
print(eras.head())
print()
print(eras.tail())

In [ ]:
# Print the percent of different values in the target column of train_df.
tgt_pct_df = (
    np.round(train_df[target].value_counts(normalize=True)  * 100.).astype(int) # convert to pct
    .reset_index()  # make series into df
    .sort_values(by=target)  # is default sorted by proportions
)
# show a bar graph of the proportions in the pandas dataframe display.
display(
    tgt_pct_df
    .style
    .bar(subset=["proportion"])
    # show only 2 decimal places in target
    .format({target: "{:.2f}"})
    .set_caption("Percent of target values in train_df")
)

## Build a learning curve

Look at the model performance as we increase the number of eras in training. Let's ensure we always include the most recent eras and walk our way back to increasing more eras.

In [ ]:
eras = train_df.erano

In [ ]:
# Plot the distribution of eras in the training data.
eras.hist(bins=30);
plt.title(f"Distribution of eras in training\nNumber of unique eras: {eras.nunique()}");
plt.xlabel("Era");
plt.ylabel("Count");
unique_eras = np.array(eras.unique())
print(unique_eras)

In [ ]:
# learning curve data
lc_data = []
era_cnts = np.linspace(10, len(unique_eras), 4).astype(int)
min_eras = [unique_eras[-era_cnt] for era_cnt in era_cnts]
print(f"{era_cnts=}")
print(f"{min_eras=}")
X_val, y_val = val_df[features].astype(float), val_df[target].astype(float)

for era_cnt, min_era in tqdm(
    reversed(list(zip(era_cnts, min_eras))),
    desc="Different number of eras in training data",
    total=len(era_cnts),
):
    print(f"Training model with {era_cnt} eras and {min_era} min era...")
    model = lgb.LGBMRegressor(
        colsample_bytree=0.006,
        learning_rate=0.14,
        max_depth=4,
        n_estimators=2000,
        n_jobs=4,
    )
    era_cnt_df = train_df[train_df.erano >= min_era]
    print("Era limited training df shape:", era_cnt_df.shape)
    X_train, y_train = era_cnt_df[features].astype(float), era_cnt_df[target].astype(float)
    model.fit(X=X_train, y=y_train)
    lc_data.append(
        {
            "min_era": min_era,
            "num_eras": era_cnt,
            "corr": nmr_metrics.correlation_score(y_true=y_val, y_pred=model.predict(X_val)),
            "model": model,
        }
    )
    print(f"Correlation score for {era_cnt=}: {lc_data[-1]['corr']}\n\n")
lc_df = pd.DataFrame(lc_data)
display(lc_df.style.bar(subset=["corr"]).format({"corr": "{:.2f}"}))

In [ ]:
plt.plot(lc_df["corr"]);
plt.title("Validation corr score vs. number of eras in training data");
plt.xlabel("Number of eras in training data");
plt.ylabel("Correlation score");
plt.grid()

In [ ]:
lc_data[0]["model"].feature_importances_

In [ ]:
# Plot the feature importances of the models for the top 5 features.

for i, lc_datum in enumerate(lc_data):
    print(f"Top 5 features for {era_cnts[i]} eras in training data")
    display(
        pd.DataFrame(
            {
                "feature": features,
                "importance": lc_datum["model"].feature_importances_,
            }
        )
        .sort_values(by="importance", ascending=False)
        .head(5)
        .style.bar(subset=["importance"])
    )

In [ ]:
cv_score = []
models = []
settings = list(
    it.product(
        [0.006, 0.01],
        [0.14],
        [4],
    ),
)

for lr, cs, md in settings:
    models.append(
        lgb.LGBMRegressor(
            colsample_bytree=cs,
            learning_rate=lr,
            n_estimators=2000,
            max_depth=md,
            n_jobs=4,
        )
    )
for model in tqdm(models):
    score = np.mean(
        model_selection.cross_val_score(
            model,
            train_df[features].astype(np.float16),
            train_df[target],
            cv=nmr_metrics.TimeSeriesSplitGroups(5),
            n_jobs=1,
            groups=eras,
            scoring=metrics.make_scorer(nmr_metrics.correlation_score, greater_is_better=True),
            error_score="raise",
        )
    )
    cv_score.append(score)
print(cv_score)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(range(len(cv_score)), np.array(cv_score))
plt.xticks(range(len(cv_score)), settings)
plt.xlabel("Setting");
plt.ylabel("CV score");
plt.title("CV score for different settings");

In [ ]:
# Train the best model on the full training data
chosen_model = models[cv_score.index(max(cv_score))]
chosen_model.fit(train_df[features].astype(np.float16), train_df[target])
display(chosen_model)

# Compute the numerai_score on the validation data
val_preds = chosen_model.predict(val_df[features].astype(np.float16))
numerai_score = nmr_metrics.numerai_score(
    y_pred=val_preds, y_true=val_df[target], eras=val_df.erano,
)

In [ ]:
explainer = shap.TreeExplainer(models[0])
shap_values = explainer(train_df[features].astype(np.float16))
shap.summary_plot(shap_values, train_df[features].astype(np.float16), plot_type="bar")